In [14]:
import numpy as np
import h5py
import scipy.stats

In [15]:
# Loading the data
with h5py.File('../HAR/preprocessed.hdf5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    y_train = np.array(hf.get('y_train'))
    s_train = np.array(hf.get('s_train'))
    x_test = np.array(hf.get('x_test'))
    y_test = np.array(hf.get('y_test'))
    s_test = np.array(hf.get('s_test'))
    x_train_with_past = np.array(hf.get('x_train_with_past'))
    y_train_with_past = np.array(hf.get('y_train_with_past'))
    x_test_with_past = np.array(hf.get('x_test_with_past'))
    y_test_with_past = np.array(hf.get('y_test_with_past'))

In [72]:
# Learning a one component Gaussian over all the features
def compute_transition(y, alpha=0.1):
    '''
    Compute the transition matrice.
    Rows: states to
    cols: states from
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    transition = alpha*np.ones((num_state, num_state))
    for i in xrange(y.shape[0]-1):
        transition[y[i+1]-1, y[i]-1] += 1
    # Normalisation (column should be normalized)
    transition /= np.sum(transition, axis=1)[:, np.newaxis]
    return transition

def compute_emission(x, y):
    '''
    Compute the parameters of the gaussian distribution
    of the emission given each state.
    We assume each emission distribution is independent,
    the covariance matrix is diagonal then.
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    
    sigma_diag = np.zeros((num_state, x.shape[1]))
    mu = np.zeros((num_state, x.shape[1]))
    for s in xrange(num_state):
        x_s = x[(y == s+1), :]
        # Computing mu_s
        mu[s] = np.mean(x_s, axis=0)
        # Computing sigma_s (by column)
        sigma_diag[s] = np.std(x_s, axis=0)

    return mu, sigma_diag

def compute_logscore(data, log_transition, mu, sigma_diag, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = compute_logB(data, mu, sigma_diag, j)

    return y + log_transition

def compute_logscore_pymc(data, log_transition, means, cov, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = scipy.stats.multivariate_normal.logpdf(data, mean=means[j], cov=covs[j])
    return y + log_transition

def viterbi(inputs, init, log_transition, mu, sigma_diag, C):
    '''
    Evaluates the highest scoring sequence
    '''
    y = np.zeros((C, C))
    initial = np.zeros(C)

    initial[init] = 1
    initial = np.log(initial)

    n = inputs.shape[0]
    # To store the maxes
    max_table = np.zeros((n, C))
    backpointer_table = np.zeros((n, C))

    # first timestep
    # the initial most likely paths are the initial state distribution
    state_init = initial + compute_logscore(inputs[0,:], log_transition, mu, sigma_diag, C)
    maxes = np.max(state_init, axis=1)
    backpointers = np.argmax(state_init, axis=1)
    max_table[0, :] = maxes

    for i in xrange(1, n):
        # COmputing the score
        y = compute_logscore(inputs[i, :], log_transition, mu, sigma_diag, C)
        scores = y + np.repeat(maxes.reshape(1, C), C, axis=0)

        # compute new maxes
        maxes = np.max(scores, axis=1)
        backpointers = np.argmax(scores, axis=1)

        max_table[i, :] = maxes
        backpointer_table[i, :] = backpointers

    # follow backpointers to recover max path
    classes = np.zeros(n)
    classes[n-1] = np.argmax(maxes, axis=0)
    for i in xrange(n-1, 0, -1):
        classes[i-1] = backpointer_table[i, classes[i]]

    return classes

def standardize(x):
    '''
    Standardize each column of x
    '''
    x_std = np.std(x, axis=0)
    x_mu = np.mean(x, axis=0)
    
    return (x - x_mu)/x_std[np.newaxis, :]

def compute_accuracy(pred_classes, true_classes):
    '''
    Compute accuracy
    '''
    return np.sum(pred_classes == true_classes) /(1.*len(pred_classes))

def compute_logB(data_point, mu, sigma_diag, j):
    '''
    Compute log(p(x|s_j))
    '''
    return np.sum([scipy.stats.norm.logpdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

def compute_B(data_point, mu, sigma_diag, j):
    '''
    Compute p(x|s_j)
    '''
    return np.prod([scipy.stats.norm.pdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

## 1) Sequence Prediction

### Subsample + MLE

In [73]:
# We retain 6 features (known to be independent)

x = np.concatenate((x_train[:, :3], x_train[:, 41:44]), axis=1)
x_sub_test = np.concatenate((x_test[:, :3], x_test[:, 41:44]), axis=1)
print(x.shape)

(7352, 6)


In [74]:
# Learning the HMM

# standardization
x_standard = standardize(x)
print(x_standard.shape)

# ### TRANSITION
transition_train = compute_transition(y_train)
log_transition_train = np.log(transition_train)
print(transition_train.shape)

# ### EMISSION
mu, sigma_diag = compute_emission(x_standard, y_train)
print(mu.shape)
print(sigma_diag.shape)

(7352, 6)
(6, 6)
(6, 6)
(6, 6)


In [75]:
%%time
# Sequence prediction
C = 6
sample_size = 3000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY train: 0.876666666667
CPU times: user 6.93 s, sys: 40.1 ms, total: 6.97 s
Wall time: 7.12 s


In [76]:
%%time
x_sub_test_standard = standardize(x_sub_test)
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY test: 0.768917543264
CPU times: user 6.73 s, sys: 32.6 ms, total: 6.76 s
Wall time: 6.84 s


In [193]:
print seq_pred_test[:100]

[ 5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


### Subsample + pyMC

In [220]:
# We use here the model fited by PyMc
means = []
covs = []

with h5py.File('../HAR/means_covs.hdf5', "r") as f:
    for k in xrange(6):
        means.append(np.array(f.get('mu_{}'.format(k+1))))
        # Pymc provides tau, cov = (tau)^{-1}
        covs.append(np.linalg.pinv(np.array(f.get('cov_{}'.format(k+1)))))
means = np.array(means)
covs = np.array(covs)

In [221]:
%%time
# Sequence prediction
C = 6
sample_size = 1000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, means, covs, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 16 µs
ACCURACY train: 0.278


### More features

In [101]:
# We retain 6 features (known to be independent)
features = [0, 1, 2, 40, 41, 42, 80, 81, 82, 120, 121, 122] + range(555, 561)
x_sub_train = x_train[:, features]
x_sub_test = x_test[:, features]
print(x_sub_train.shape)

(7352, 18)


In [106]:
# Learning the HMM

# standardization
x_standard = standardize(x_sub_train)
print(x_standard.shape)

# ### TRANSITION
transition_train = compute_transition(y_train)
log_transition_train = np.log(transition_train)
print(transition_train.shape)

# ### EMISSION
mu, sigma_diag = compute_emission(x_standard, y_train)
print(mu.shape)
print(sigma_diag.shape)

(7352, 18)
(6, 6)
(6, 18)
(6, 18)


In [109]:
%%time
# Sequence prediction
C = 6
sample_size = 3000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY train: 0.851666666667
CPU times: user 21.4 s, sys: 231 ms, total: 21.6 s
Wall time: 23.1 s


In [110]:
%%time
x_sub_test_standard = standardize(x_sub_test)
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY test: 0.772310824567
CPU times: user 20.2 s, sys: 163 ms, total: 20.4 s
Wall time: 21.2 s


# Forward Backward algorithm

In [28]:
print(compute_B(x[0], mu, sigma_diag, 4))
print(compute_logB(x[0], mu, sigma_diag, 4))

0.0364018687618
-3.31313516605


In [77]:
def forward(x, init, end, log_transition, mu, sigma_diag):
    '''
    Compute log(p(X|lambda)) with lambda the HMM parameters
    (here transition, mu, sigma_diag)
    alpha[t, j] = p(x_1, ... x_t, S(t) = s_j|lambda)
    NB:
        alpha[0, :] used only as initialization
        log-sum-exp trick used
    '''
    C = mu.shape[0]
    T = x.shape[0]
    
    # Initialization
    alpha = np.zeros((T+1, C))
    alpha[0, init] = 1
    alpha[0,:] = np.log(alpha[0,:])
    
    # Recursion
    for t in xrange(1, T+1):
        
        for j in xrange(C):
            # Compute log p(S(t)|x_1, ..., x_{t-1})
            a = alpha[t-1, :] + log_transition[j,:]
            # Compute log evidence at time t in [1, T], array index shifted of 1
            b_t_j = compute_logB(x[t-1], mu, sigma_diag, j)
            a += b_t_j
            # log-sum-exp trick
            max_a = np.max(a)
            alpha[t, j] = max_a + np.log(np.sum(np.exp(a - max_a)))
        # Normalization
        Z_t = np.sum(np.exp(alpha[t, :]))
        alpha[t,:] -= np.log(Z_t) 
    
    # Termination
    #b_t_j = compute_logB(x[T-1], mu, sigma_diag, end)
    #a = b_t_j + alpha[T, :] + log_transition[:,end]
    #max_a = np.max(a)
    #alpha[T, end] = max_a + np.log(np.sum(np.exp(a - max_a)))
    # Normalization
    #Z_T = np.sum(np.exp(alpha[, :]))
    #alpha[t,:] -= np.log(Z_t)
    
    return alpha

def backward(x, init, end, log_transition, mu, sigma_diag):
    '''
    Compute the log backward probabilities.
    beta[t, j] = p(x_{t+1}, ..., x_{T}| S(t) = s_j, lambda)
    '''
    C = mu.shape[0]
    T = x.shape[0]
    # Initialization
    beta = np.zeros((T+1, C))
    for i in xrange(C):
        beta[T, i] = log_transition[end, i]
    
    # Recursion
    for t in xrange(T-1, 0, -1):
        for j in xrange(C):
            a = beta[t+1, :] + log_transition[:,j]
            for i in xrange(C):
                b_t_i = compute_logB(x[t], mu, sigma_diag, i)
                a[i] += b_t_i
            max_a = np.max(a)
            beta[t, j] = max_a + np.log(np.sum(np.exp(a - max_a))) 
    
    # Termination
    a = beta[1, :] + log_transition[:, init]
    for i in xrange(C):
        b_t_i = compute_logB(x[0], mu, sigma_diag, i)
        a[i] += b_t_i
    max_a = np.max(a)
    beta[0, init] = max_a + np.log(np.sum(np.exp(a - max_a))) 
        
    return beta

def compute_state_probability(alpha, beta):
    '''
    compute state occupation log probability
    '''
    # Removing the initialization
    gamma = alpha[1:, :] + beta[:-1, :]
    # Normalization with log-sum-exp trick
    for t in xrange(gamma.shape[0]):
        row_max = np.max(gamma[t, :])
        Z_row = row_max + np.log(np.sum(np.exp(gamma[t, :] - row_max)))
        gamma[t, :] -= Z_row
    return gamma

def compute_state_transition(x, alpha, beta, log_transition, mu, sigma_diag):
    '''
    Compute log(p(S(t) = s_i, S(t+1) = s_j| X, lambda))
    '''
    T = x.shape[0]
    C = mu.shape[0]
    psi = np.zeros((T, C, C))
    for t in xrange(T):
        for j in xrange(C):
            b_t_j = compute_logB(x[t], mu, sigma_diag, j)
            for i in xrange(C):
                psi[t, i, j] = alpha[t, i] + log_transition[j, i] + beta[t+1, j] + b_t_j
        # Normalization with log sum exp trick
        mat_max = np.max(psi[t, :, :])
        Z_mat = mat_max + np.log(np.sum(np.exp(psi[t, :, :] - mat_max)))
        psi[t, :, :] -= Z_mat
    
    return psi
    
    
def forward_backward(x, init, end, log_transition, mu, sigma_diag, n_iterations):
    '''
    EM algorithm to fit an HMM to a sequence of observation.
    Take as argnument an initial HMM and returns a finer one.
    '''
    T = x.shape[0]
    C = mu.shape[0]
    
    # Copying object
    mu = mu.copy()
    sigma_diag = sigma_diag.copy()
    log_transition = log_transition.copy()
    
    for it in xrange(n_iterations):
        # ##### E-step: estimate the state occupation probabilities (in LOG)
        log_alpha = forward(x, init, end, log_transition, mu, sigma_diag)
        log_beta = backward(x, init, end, log_transition, mu, sigma_diag)
        log_state_probability = compute_state_probability(log_alpha, log_beta)
        log_state_transition = compute_state_transition(x, log_alpha, log_beta, log_transition, mu, sigma_diag)

        # ##### M-step: re-estimate HMM parameers
        state_probability = np.exp(log_state_probability)
        
        # Mu
        denom = np.sum(state_probability, axis=0)[np.newaxis, :]
        mu = np.dot(state_probability.T, x)/denom
    
        # Sigma_diag (stands for the standard deviation)
        for j in xrange(C):
            diff = x - mu[j]
            numerator = 0
            for t in xrange(T):
                numerator += state_probability[t, j]*(diff[t, :])**2
            sigma_diag[:, j] = numerator
        sigma_diag /= denom
        # Standard deviation from the variance
        sigma_diag = np.sqrt(sigma_diag)
        
        # Transition 
        state_transition = np.exp(log_state_transition)
        for j in xrange(C):
            for i in xrange(C):
                log_transition[i, j] = np.log(np.sum(state_transition[:, i, j]))
            # Normalization
            log_transition[:,j] -= np.log(np.sum(np.exp(log_transition[:, j])))
    
    return log_transition, mu, sigma_diag
    

In [389]:
log_transition_train

array([[-0.03526312, -9.41458649, -9.41458649, -9.41458649, -9.41458649,
        -3.37195365],
       [-9.28042598, -0.05116572, -2.9870067 , -9.28042598, -9.28042598,
        -9.28042598],
       [-3.15421695, -4.24808989, -0.05897258, -9.19684978, -9.19684978,
        -9.19684978],
       [-9.46234345, -9.46234345, -9.46234345, -0.03439482, -3.41971062,
        -7.06444818],
       [-9.52850315, -3.55979559, -6.48398071, -9.52850315, -0.03140614,
        -9.52850315],
       [-9.5522265 , -9.5522265 , -9.5522265 , -3.48611841, -9.5522265 ,
        -0.03139126]])

In [489]:
%%time
alpha = forward(x_standard, 4, 1, log_transition_train, mu, sigma_diag)

CPU times: user 18.7 s, sys: 144 ms, total: 18.9 s
Wall time: 20 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: divide by zero encountered in log


In [490]:
alpha

array([[            -inf,             -inf,             -inf,
                    -inf,   0.00000000e+00,             -inf],
       [ -9.80446686e+00,  -1.39896121e+01,  -1.19783671e+01,
         -7.03481900e+00,  -9.43447673e-04,  -2.00466250e+01],
       [ -1.03542142e+01,  -1.45166617e+01,  -1.26468058e+01,
         -7.16116529e+00,  -8.12053327e-04,  -2.00159851e+01],
       ..., 
       [ -7.08676390e+00,  -1.94890746e+00,  -1.83726911e+00,
         -8.44569428e+00,  -3.60585802e-01,  -1.91747196e+01],
       [ -6.57819792e+00,  -4.20977316e+00,  -3.55021758e+00,
         -8.09704220e+00,  -4.63192843e-02,  -2.03699196e+01],
       [ -6.09512926e+00,  -1.44331336e+00,  -3.36134569e-01,
         -5.10653272e+00,  -3.19374796e+00,  -1.49951343e+01]])

In [493]:
%%time
beta = backward(x_standard, 4, 1, transition_train, mu, sigma_diag)

CPU times: user 1min 56s, sys: 1.67 s, total: 1min 58s
Wall time: 2min 15s


In [494]:
beta

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,  -2.82025248e+04,   0.00000000e+00],
       [ -2.82035546e+04,  -2.82036674e+04,  -2.82036782e+04,
         -2.82036825e+04,  -2.82027995e+04,  -2.82036924e+04],
       [ -2.82040604e+04,  -2.82041555e+04,  -2.82041677e+04,
         -2.82041713e+04,  -2.82032789e+04,  -2.82041812e+04],
       ..., 
       [ -3.45021693e+00,  -3.91616739e+00,  -3.88044103e+00,
         -4.00569347e+00,  -3.51574398e+00,  -3.99576295e+00],
       [ -3.53398986e+00,  -2.93914639e+00,  -3.08190562e+00,
         -3.58311880e+00,  -3.58653686e+00,  -3.58678986e+00],
       [  9.32314003e-05,   9.50121201e-01,   5.04381876e-02,
          9.32314003e-05,   9.32314003e-05,   9.32314003e-05]])

In [536]:
%%time
log_state_probability = compute_state_probability(alpha, beta)

CPU times: user 81.4 ms, sys: 2.41 ms, total: 83.8 ms
Wall time: 84.8 ms


In [537]:
log_state_probability

array([[ -2.83802551e+00,  -7.02317075e+00,  -5.01192578e+00,
         -6.83776589e-02,  -2.81955593e+04,  -1.30801836e+01],
       [ -1.11088726e+01,  -1.53841452e+01,  -1.35250562e+01,
         -8.04372249e+00,  -3.37693429e-04,  -2.09084622e+01],
       [ -1.15404438e+01,  -1.54912197e+01,  -1.36921102e+01,
         -8.12898073e+00,  -3.05963196e-04,  -2.10565007e+01],
       ..., 
       [ -7.02242111e+00,  -2.36639478e+00,  -2.22747367e+00,
         -8.90506017e+00,  -2.26456718e-01,  -1.96242644e+01],
       [ -6.49887639e+00,  -4.59640208e+00,  -3.90112015e+00,
         -8.57319719e+00,  -3.25247973e-02,  -2.08361441e+01],
       [ -6.56384937e+00,  -1.31719001e+00,  -3.52770436e-01,
         -5.62438178e+00,  -3.71501507e+00,  -1.55166544e+01]])

In [518]:
%%time
log_state_transition = compute_state_transition(x_standard, alpha, beta, log_transition_train, mu, sigma_diag)

CPU times: user 17.6 s, sys: 152 ms, total: 17.8 s
Wall time: 18.2 s


In [525]:
log_state_transition

array([[[            -inf,             -inf,             -inf,
                     -inf,             -inf,             -inf],
        [            -inf,             -inf,             -inf,
                     -inf,             -inf,             -inf],
        [            -inf,             -inf,             -inf,
                     -inf,             -inf,             -inf],
        [            -inf,             -inf,             -inf,
                     -inf,             -inf,             -inf],
        [ -1.05590495e+01,  -1.48570199e+01,  -1.28565418e+01,
          -7.91730049e+00,  -3.93382266e-04,  -2.09390263e+01],
        [            -inf,             -inf,             -inf,
                     -inf,             -inf,             -inf]],

       [[ -1.20633856e+01,  -2.52092080e+01,  -1.73755579e+01,
          -2.39249540e+01,  -1.93009548e+01,  -3.10434321e+01],
        [ -2.56278542e+01,  -2.01650930e+01,  -2.26545761e+01,
          -2.81100993e+01,  -1.75173925e+01,  

In [538]:
state_probability = np.exp(log_state_probability)
    
denom = np.sum(state_probability, axis=0)[np.newaxis, :]
mu_new = np.dot(state_probability.T, x_standard)/denom

In [588]:
mu_new

array([[ 0.03356389,  0.01105426,  0.01057963, -0.71033201, -0.36212407,
        -0.15569301],
       [-0.11891079, -0.17461568, -0.15459098, -0.96517308, -0.62964239,
         0.20831907],
       [ 0.11522654, -0.01146936,  0.01763806, -0.45692365, -0.18504464,
         0.09718053],
       [ 0.03289123,  0.06227521,  0.00377912,  0.35632647,  0.158786  ,
        -0.14840064],
       [ 0.06208901,  0.02523136,  0.01186778, -0.75999966, -0.30079598,
        -0.39557379],
       [-0.12485988,  0.0875242 ,  0.11072639,  2.53610192,  1.31882108,
         0.39416784]])

In [589]:
mu

array([[ 0.02522293, -0.00177132,  0.0045369 , -0.56560315, -0.45669749,
        -0.16650716],
       [-0.1787534 , -0.21936092, -0.19922794, -0.80040918, -0.69653595,
         0.23307432],
       [ 0.1947258 ,  0.03247523,  0.05793074, -0.47849408, -0.42016397,
         0.20147609],
       [-0.01479045,  0.13607265,  0.04520951,  0.30031038,  0.1961177 ,
        -0.18645536],
       [ 0.06840119,  0.03853049,  0.03198569, -0.5189657 , -0.34882962,
        -0.30448021],
       [-0.07539678, -0.01592395,  0.03482719,  1.67087662,  1.38497542,
         0.29390974]])

In [566]:
sigma_diag_new = np.zeros((C, C))
for j in xrange(C):
    diff = x_standard - mu_new[j]
    sigma_diag_new[:, j] = np.std(diff * np.sqrt(np.abs(state_probability[:,j]))[:, np.newaxis], axis=0)
sigma_diag_new /= denom

In [584]:
sigma_diag_new2 = np.zeros((C, C))
for j in xrange(C):
    diff = x_standard - mu_new[j]
    numerator = 0
    for t in xrange(x_standard.shape[0]):
        numerator += state_probability[t, j]*(diff[t, :])**2
    sigma_diag_new2[:, j] = numerator
sigma_diag_new2 /= denom
sigma_diag_new2 = np.sqrt(sigma_diag_new2)

In [572]:
t = 0
j = 0
state_probability[t, j]*(diff[t, :])**2

array([ 0.0062025 ,  0.00133846,  0.01646623,  0.50740366,  0.09251682,
        0.02549438])

In [573]:
state_probability[t, j]

0.058541140568316746

In [574]:
(diff[t, :])**2

array([ 0.10595119,  0.0228635 ,  0.28127626,  8.66747144,  1.58037271,
        0.43549506])

In [585]:
sigma_diag

array([[ 0.71640716,  0.51146808,  0.57251528,  0.2204994 ,  0.46464535,
         0.16410181],
       [ 1.11010689,  0.90720976,  1.06258925,  0.2658547 ,  0.57818387,
         0.51386202],
       [ 1.35294292,  0.66268894,  0.89404002,  0.24217263,  0.4707116 ,
         0.37867326],
       [ 0.59754132,  0.79416585,  0.80001113,  0.5011433 ,  0.6398332 ,
         0.79073217],
       [ 0.28594946,  0.43716819,  0.6298047 ,  0.27047212,  0.46420367,
         0.3867504 ],
       [ 1.44478089,  1.80077441,  1.5841213 ,  0.84965169,  1.13787962,
         1.96969504]])

In [586]:
sigma_diag_new

array([[  2.30554439e-04,   4.10772398e-04,   4.95822233e-04,
          1.41061848e-04,   3.30909761e-05,   4.59421014e-04],
       [  1.55652653e-04,   3.17420743e-04,   2.63945856e-04,
          1.91344261e-04,   7.26296199e-05,   5.69999220e-04],
       [  1.77785442e-04,   3.82485452e-04,   3.53593949e-04,
          2.06670133e-04,   9.94751635e-05,   5.16440810e-04],
       [  7.42602795e-05,   7.94035152e-05,   7.53626986e-05,
          1.69745702e-04,   1.13601560e-04,   3.27960533e-04],
       [  1.52734624e-04,   2.44624221e-04,   1.53729793e-04,
          2.39289368e-04,   1.33442341e-04,   2.81330024e-04],
       [  4.55649733e-05,   1.90327290e-04,   1.15648745e-04,
          1.46776860e-04,   3.88517025e-05,   6.10417457e-04]])

In [587]:
sigma_diag_new2

array([[ 0.69803711,  1.17902575,  1.28559381,  0.37834656,  0.11370342,
         1.52138601],
       [ 0.47126296,  0.91111854,  0.68439498,  0.51317881,  0.24914702,
         1.8875513 ],
       [ 0.53827752,  1.09797993,  0.9168255 ,  0.55427504,  0.34122698,
         1.71029581],
       [ 0.23535667,  0.23916148,  0.197197  ,  0.45556115,  0.41684469,
         1.16538881],
       [ 0.46526612,  0.70848496,  0.40628037,  0.64280809,  0.45784718,
         0.93218481],
       [ 0.13845166,  0.54662518,  0.30018854,  0.39455191,  0.13598366,
         2.0213701 ]])

In [61]:
for j in xrange(C):
    log_transition_train2[:,j] -= np.log(np.sum(np.exp(log_transition_train2[:, j])))

In [581]:
log_transition

array([[ -3.64585122, -11.94525074,  -6.96923301, -15.5181864 ,
        -13.72170815,  -9.3609343 ],
       [-12.63756534,  -3.75222689,  -8.71952589, -15.27760616,
         -7.18035477,  -9.42512237],
       [-12.59086519,  -7.00424069,  -3.96467589, -15.77714722,
        -10.47246074, -10.66642844],
       [-13.08084501, -13.74388952, -14.0022585 ,  -3.8912238 ,
        -12.52021385,  -6.80869384],
       [-13.80832909,  -9.28884759, -11.73486159,  -7.02889944,
         -3.40324794,  -7.61915268],
       [ -6.98018803, -10.52376177, -11.10211928,  -9.40446002,
        -12.65426829,  -3.50104651]])

In [582]:
log_transition_train

array([[-0.03526312, -9.41458649, -9.41458649, -9.41458649, -9.41458649,
        -3.37195365],
       [-9.28042598, -0.05116572, -2.9870067 , -9.28042598, -9.28042598,
        -9.28042598],
       [-3.15421695, -4.24808989, -0.05897258, -9.19684978, -9.19684978,
        -9.19684978],
       [-9.46234345, -9.46234345, -9.46234345, -0.03439482, -3.41971062,
        -7.06444818],
       [-9.52850315, -3.55979559, -6.48398071, -9.52850315, -0.03140614,
        -9.52850315],
       [-9.5522265 , -9.5522265 , -9.5522265 , -3.48611841, -9.5522265 ,
        -0.03139126]])

### TEST FB

In [93]:
%%time
init = 4
end = 1
n_iterations = 1
log_transition_train3, mu3, sigma_diag3 = forward_backward(x_standard, init, end, log_transition_train2, mu2, sigma_diag2, n_iterations)

CPU times: user 2min 26s, sys: 1.66 s, total: 2min 28s
Wall time: 2min 41s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: divide by zero encountered in log


In [95]:
# Comparing value
print mu
print mu2
print mu3

[[ 0.02522293 -0.00177132  0.0045369  -0.56560315 -0.45669749 -0.16650716]
 [-0.1787534  -0.21936092 -0.19922794 -0.80040918 -0.69653595  0.23307432]
 [ 0.1947258   0.03247523  0.05793074 -0.47849408 -0.42016397  0.20147609]
 [-0.01479045  0.13607265  0.04520951  0.30031038  0.1961177  -0.18645536]
 [ 0.06840119  0.03853049  0.03198569 -0.5189657  -0.34882962 -0.30448021]
 [-0.07539678 -0.01592395  0.03482719  1.67087662  1.38497542  0.29390974]]
[[ 0.03047047  0.00614208  0.01634892 -0.75639405 -0.32594792 -0.15097713]
 [-0.11167925 -0.16113411 -0.1574661  -1.09799345 -0.60916783  0.23507314]
 [ 0.13023048 -0.00384958  0.01150092 -0.4600853  -0.15943268  0.10107851]
 [ 0.01879315  0.04794091 -0.018312    0.43814374  0.18712404 -0.14245351]
 [ 0.06033065  0.02498768  0.00380992 -0.83389237 -0.30337591 -0.43920883]
 [-0.12814551  0.08591302  0.14411833  2.71022143  1.21080031  0.39648781]]
[[ -4.05113750e-02  -8.49775267e-02   4.19586273e-02  -3.34241018e-01
   -1.61918132e-01   3.57223

In [96]:
print sigma_diag
print sigma_diag2
print sigma_diag3

[[ 0.71640716  0.51146808  0.57251528  0.2204994   0.46464535  0.16410181]
 [ 1.11010689  0.90720976  1.06258925  0.2658547   0.57818387  0.51386202]
 [ 1.35294292  0.66268894  0.89404002  0.24217263  0.4707116   0.37867326]
 [ 0.59754132  0.79416585  0.80001113  0.5011433   0.6398332   0.79073217]
 [ 0.28594946  0.43716819  0.6298047   0.27047212  0.46420367  0.3867504 ]
 [ 1.44478089  1.80077441  1.5841213   0.84965169  1.13787962  1.96969504]]
[[ 0.70750722  1.17713363  1.30456959  0.34256486  0.10371567  1.54198509]
 [ 0.48415749  0.92615446  0.68304973  0.47812167  0.2598459   1.91619079]
 [ 0.55127949  1.13025333  0.92465242  0.5387193   0.35923517  1.71270929]
 [ 0.26091919  0.31782402  0.19818077  0.44055814  0.49423994  1.24849524]
 [ 0.46654296  0.71487613  0.41514744  0.6515822   0.45241921  0.93958793]
 [ 0.13857153  0.56400559  0.29402873  0.38163688  0.15518971  2.05285062]]
[[ 1.46517597  0.99919067  1.93199114  0.23161072  0.57908296  0.19392374]
 [ 0.96855566  0.759652

In [97]:
print log_transition_train
print log_transition_train2
print log_transition_train3

[[-0.03526312 -9.41458649 -9.41458649 -9.41458649 -9.41458649 -3.37195365]
 [-9.28135786 -0.0520976  -2.98793858 -9.28135786 -9.28135786 -9.28135786]
 [-3.15421695 -4.24808989 -0.05897258 -9.19684978 -9.19684978 -9.19684978]
 [-9.46234345 -9.46234345 -9.46234345 -0.03439482 -3.41971062 -7.06444818]
 [-9.5277754  -3.55906784 -6.48325296 -9.5277754  -0.03067839 -9.5277754 ]
 [-9.5522265  -9.5522265  -9.5522265  -3.48611841 -9.5522265  -0.03139126]]
[[ -0.0418062   -7.81261533  -2.99537544 -12.09935335  -8.31612631
   -5.89637263]
 [ -5.87076454  -0.06279971  -4.60938851 -10.8437171   -3.44226665
   -6.16112401]
 [ -5.48595785  -3.21253275  -0.06482923 -10.46997093  -6.3174197
   -7.16918736]
 [ -7.80561011  -6.46602867  -6.11347767  -0.07136139  -4.81505057
   -3.43292002]
 [ -9.34441109  -4.2486709   -8.49497209  -3.07448421  -0.04684031
   -4.32094817]
 [ -3.39668156  -5.43282669  -7.87687275  -3.78946289  -5.623438
   -0.05256675]]
[[-0.04182487 -4.13277784 -3.924197   -5.64038402 -5.

In [99]:
%%time
# Sequence prediction after 1 iteration
C = 6
sample_size = 3000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train3, mu3, sigma_diag3, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY train: 0.319666666667
CPU times: user 6.95 s, sys: 43.7 ms, total: 6.99 s
Wall time: 7.12 s


In [71]:
np.sum(np.exp(log_transition_train), axis=0)

array([ 1.00833793,  0.99308764,  0.99492882,  0.99715832,  1.00215098,
        1.00454116])

In [69]:
np.sum(np.array([[1, 2, 3], [1, 2, 3]]), axis=1)

array([6, 6])